In [55]:
%matplotlib inline

import numpy as np
import nltk as nltk #nlp module
import matplotlib as plot #visuals
import sklearn as skl
import pandas as pd
import spacy #nlp module
import re
import random
from random import sample
from collections import defaultdict, Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cross_validation import train_test_split
from nltk import sent_tokenize, word_tokenize, pos_tag
from nltk.corpus import treebank
from nltk.corpus import state_union
from nltk.tokenize import PunktSentenceTokenizer


nlp = spacy.load('en')
from spacy.en import English
parser = English()

Read in the CSV and name the columns. We're working mainly with the third column - summaries about the articles. 

In [27]:
textRead = pd.read_csv('factCheck.csv')
#textRead = textRead.parse('Sheet1')
textRead.columns = ['Json', 'Accuracy', 'Title', 'Genre', 'KeywordName', 'Occupation', 'Location', 'PoliticalParty', 
                    '1', '2', '3', '4', '5', 'Source', 'Url']


In [28]:
sentenceFrame = textRead[textRead.columns[2]]
#sentenceFrame = pd.DataFrame(sentenceFrame)

Print the "-ing" verbs in the frame. Not really sure why. But here they are! 

In [31]:
for line in sentenceFrame:
    for word in line.split():
        if word.endswith('ing'):
            print (word)

voting
starting
during
spending
making
cutting
Shooting
endorsing
going
going
existing
including
seeking
making
having
going
choosing
being
learning
being
driving
During
forthcoming
manufacturing
nothing
fastest-growing
kidnapping
peacekeeping
setting
privatizing
facing
leaving
during
totaling
Raising
engaging
opening
going
saying
growing
receiving
looking
creating
serving
spending
getting
creating
bring
amazing
going
bring
arriving
spending
begging
waiting


In [97]:
def stem(word):
    for suffix in ['ing', 'ly', 'ed', 'ious', 'ies', 'ive', 'es', 's', 'ment']:
        if word.endswith(suffix): return word[:-len(suffix)]
        return word

In [6]:
sorted(set(sentenceFrame))
len(set(sentenceFrame))


149

Lexical diversity is how common most of the words are, to see if some words appear more than others. In this case, since the diversity returns 1, most of the words are unique. Nothing special. 

In [21]:
def lexical_diversity(sentenceFrame):
    return (len(sentenceFrame)) / len(set(sentenceFrame))

lexical_diversity(sentenceFrame)

1.0

In [8]:
tokens = set(sentenceFrame)
tokens = sorted(tokens)


In [163]:
def find_features(textRead):
    words = word_tokenize(textRead)
    features = {}
    for w in words_features:
        features[w] = (w in words)
        print (features)

We start manipulating the data. First, we must tokenize the data frame to differentiate between the sentences and phrases. Stop words (common words such as "at", "by", etc.) have been removed. 

In [29]:
txt = sentenceFrame.apply(word_tokenize)
txt.head()

0    [When, did, the, decline, of, coal, start, ?, ...
1    [Hillary, Clinton, agrees, with, John, McCain,...
2    [Health, care, reform, legislation, is, likely...
3    [The, economic, turnaround, started, at, the, ...
4    [The, Chicago, Bears, have, had, more, startin...
Name: Title, dtype: object

In [132]:
with open("stop_words.txt", 'r') as f:
    stopwords = f.read().split("\n")

filtered_sentence = [w for w in txt if not w in stopwords]

filtered_sentence = []

for w in txt:
    if w not in stopwords:
        filtered_sentence.append(w)

#print(txt)
#print(filtered_sentence)
filtered_sentence = pd.DataFrame(filtered_sentence)
#filtered_sentence

Here I am experimenting with different ways to create training and test sets. None are necessarily wrong, but I think the simplest way is to just subset. See line 166. 

In [69]:
def make_sets(filtered_sentence, test_portion):
    tot_ix = range(len(filtered_sentence))
    test_ix = sorted(random.sample(tot_ix, int(test_portion * len(filtered_sentence))))
    train_ix = list(set(tot_ix) ^ set(test_ix))

    test_df = filtered_sentence.ix[test_ix]
    train_df = filtered_sentence.ix[train_ix]

    return train_df, test_df


train_df, test_df = make_sets(filtered_sentence, 0.2)

In [150]:
traindf = train_df.all()
testdf = test_df.all()


In [101]:
#custom_sent_tokenizer = PunktSentenceTokenizer(traindf)
#tokenized = custom_sent_tokenizer.tokenize(testdf)

In [166]:
train = filtered_sentence[:101]
test = filtered_sentence[101:]

Now I'm going to try to apply the algorithm. We are going to use the Naive Bayes classification algorithm. 
This is a pretty popular algorithm used in text classification, implementing supervised machine learning. We're showing the machine data, and telling it "hey, this data is positive," or "this data is negative." Then, after that training is done, we show the machine some new data and ask the computer, based on what we taught the computer before, what the computer thinks the category of the new data is.

I get stuck here. Why isn't my dataframe iterable? Any suggestions?

In [167]:
classifier = nltk.NaiveBayesClassifier.train(train)

TypeError: 'numpy.int64' object is not iterable